In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Concatenate, Flatten, Embedding, Dot


# Convert user_id and item_id to categorical data
train_ratings_df['userid'] = train_ratings_df['userid'].astype('category').cat.codes.values
train_ratings_df['itemid'] = train_ratings_df['itemid'].astype('category').cat.codes.values

test_ratings_df = pd.read_csv('data/test_ratings.csv')
test_ratings_df['userid'] = test_ratings_df['userid'].astype('category').cat.codes.values
test_ratings_df['itemid'] = test_ratings_df['itemid'].astype('category').cat.codes.values

num_health_features = 13
health_features_df = pd.read_csv('data/pp_recipes.csv')
health_features_df=health_features_df.values

def generate_pairs(df):
    pairs = []
    labels = []
    grouped = df.groupby('userid')
    for user, group in grouped:
        items = group['itemid'].values
        ratings = group['rating'].values
        for i in range(len(items)):
            for j in range(len(items)):
                if ratings[i] > ratings[j]:
                    pairs.append((user, items[i], items[j]))
                    labels.append(1)
                elif ratings[i] < ratings[j]:
                    pairs.append((user, items[j], items[i]))
                    labels.append(0)                
    return np.array(pairs), np.array(labels)
train_pairs, train_labels = generate_pairs(train_ratings_df)
test_pairs, test_labels = generate_pairs(test_ratings_df)

# Define model parameters
embedding_size = 30
# Inputs for users and items
user_input = Input(shape=(embedding_size,), name='user_input')
item_input_pos = Input(shape=(embedding_size,), name='item_input_pos')
item_input_neg = Input(shape=(embedding_size,), name='item_input_neg')
item_input_health = Input(shape=(num_health_features,), name='item_input_health')
# Compute dot product of user and item features
pos_dot = Dot(axes=-1)([user_input, item_input_pos])
neg_dot = Dot(axes=-1)([user_input, item_input_neg])
#health_dot = Dot(axes=-1)([user_input, item_input_health])
# Concatenate and apply softmax
concatenated = Concatenate()([pos_dot, neg_dot, item_input_health])
output = Dense(2, activation='softmax')(concatenated)
# Build and compile the model
model = Model(inputs=[user_input, item_input_pos, item_input_neg, item_input_health], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Prepare the input data
train_user_features = user_ratings_features[train_pairs[:, 0]]
train_item_features_pos = item_ratings_features[train_pairs[:, 1]]
train_item_features_neg = item_ratings_features[train_pairs[:, 2]]

# Train the model
history = model.fit(
    [train_user_features, train_item_features_pos, train_item_features_neg, health_features_df], 
    train_labels, 
    epochs=2, 
    batch_size=64, 
    validation_split=0.1,
    verbose=1
)

test_user_features = user_ratings_features[test_pairs[:, 0]]
test_item_features_pos = item_ratings_features[test_pairs[:, 1]]
test_item_features_neg = item_ratings_features[test_pairs[:, 2]]

# Evaluate the model
predictions = model.predict(
    [test_user_features, test_item_features_pos, test_item_features_neg, health_features_df], 
    test_labels,
    verbose=1
)